In [1]:
!conda install -y -c python3 cartopy
!pip install geoplot pygeohash geopy shapely geos geopandas descartes 

/bin/bash: conda: command not found
     |████████████████████████████████| 400 kB 11.7 MB/s 
     |████████████████████████████████| 994 kB 49.4 MB/s 
     |████████████████████████████████| 12.1 MB 127 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.6 MB 13.7 MB/s 
     |████████████████████████████████| 219 kB 63.3 MB/s 
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6168 sha256=b38cf3a6cbd0020a73b7b2859c459cccf810bca8eeb3c68738f67130fe9acefa
  Stored in directory: /root/.cache/pip/wheels/e2/ed/86/be7141d743cfb4aebac76406187154cd78fe9e6535d872187a
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516274 sha256=01795b4bad9bfc82a7a16d5ec321b406deefe03fed4ce9616d40044e69a

In [2]:
from geopy import distance as geodist
import pygeohash as pgh
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import base64
from shapely.geometry import Point, Polygon

In [13]:
date_parser = lambda x: pd.to_datetime(x)

In [16]:
voyages = pd.read_csv('./voyages.csv', parse_dates=['begin_date', 'end_date'], date_parser=date_parser)
voyages.head()

,vessel,begin_date,end_date,begin_port_id,end_port_id
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0


Calculate time spent at port

In [272]:
time_spent = []

for i in range(len(voyages)):
    time_spent.append(voyages['end_date'].iloc[i] - voyages['begin_date'].iloc[i])

voyages = voyages.assign(time_spent = time_spent)
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance,efficiency,heading
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113,518.949646,255.0
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113,293.607253,213.0
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134,532.239424,91.0
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134,552.632443,214.0
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161,438.518873,246.0
...,...,...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977,615.578920,337.0
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977,189.371118,228.0
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907,223.312041,278.0
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542,183.540706,337.0


Calculate distance between two successive ports

In [109]:
distance = []
ports = pd.read_csv('./ports.csv', sep='\t')
ports = ports.reset_index(drop=True)
for i in range(len(voyages)):
  end_port = int(voyages['end_port_id'].iloc[i])
  begin_port = int(voyages['begin_port_id'].iloc[i])

  e_lat = ports.loc[ports['port'] == end_port].iloc[0][1]
  e_long = ports.loc[ports['port'] == end_port].iloc[0][2]
  b_lat = ports.loc[ports['port'] == begin_port].iloc[0][1]
  b_long = ports.loc[ports['port'] == begin_port].iloc[0][2]

  distance.append(geodist.distance((e_lat, e_long), (b_lat, b_long)).kilometers)

voyages = voyages.assign(distance = distance)
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161
...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542


Calculate efficiency

In [115]:
efficiency = []
for i in range(len(voyages)):
  time = (voyages['time_spent'].iloc[i]) / pd.offsets.Day()
  efficiency.append(voyages['distance'].iloc[i]/time)

voyages = voyages.assign(efficiency = efficiency)
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance,efficiency
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113,518.949646
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113,293.607253
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134,532.239424
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134,552.632443
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161,438.518873
...,...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977,615.578920
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977,189.371118
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907,223.312041
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542,183.540706


In [191]:
heading = []
tracking = pd.read_csv('./tracking.csv', sep='\t', parse_dates=['datetime'], date_parser=date_parser)

In [192]:
tracking = tracking.sort_values(['vessel', 'datetime'])
tracking

,vessel,datetime,lat,long,heading,speed,draft
629483,1,2019-01-01 01:53:00,29.743000,-93.869500,214.0,NaN,9.6
629484,1,2019-01-01 02:47:00,29.743000,-93.869500,214.0,0.0,9.6
629485,1,2019-01-01 05:47:00,29.743000,-93.869500,214.0,0.0,9.6
629486,1,2019-01-01 06:59:00,29.743000,-93.869500,214.0,0.0,9.6
629487,1,2019-01-01 08:53:00,29.743000,-93.869500,214.0,0.0,9.6
...,...,...,...,...,...,...,...
792015,176,2019-12-23 17:29:00,51.888150,1.672528,199.0,0.6,10.9
792017,176,2019-12-23 18:53:00,51.889468,1.674938,220.0,0.1,10.9
792018,176,2019-12-23 18:59:00,51.889498,1.674873,217.0,0.0,10.9
792019,176,2019-12-23 18:59:00,51.889498,1.674873,217.0,0.0,10.9


In [202]:
for i in range(len(voyages)):
  vessel = voyages['vessel'].iloc[i]
  end_date = voyages['end_date'].iloc[i]
  newtracking = tracking.loc[tracking['vessel'] == vessel]
  newtracking = newtracking.loc[newtracking['datetime'] == end_date]
  heading.append(newtracking.iloc[0][4])

voyages['heading'] = heading
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance,efficiency,heading
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113,518.949646,255.0
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113,293.607253,213.0
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134,532.239424,91.0
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134,552.632443,214.0
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161,438.518873,246.0
...,...,...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977,615.578920,337.0
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977,189.371118,228.0
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907,223.312041,278.0
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542,183.540706,337.0


In [210]:
voyages = voyages.reset_index(drop=True)
voyages.to_csv('dataset.csv')

In [211]:
corr = voyages.corr()
corr.style.background_gradient(cmap='coolwarm')

,vessel,begin_port_id,end_port_id,distance,efficiency,heading
vessel,1.000000,-0.033968,-0.034675,-0.426846,-0.056452,0.041264
begin_port_id,-0.033968,1.000000,-0.072412,0.039465,0.019398,-0.022829
end_port_id,-0.034675,-0.072412,1.000000,0.006420,-0.071339,0.121177
distance,-0.426846,0.039465,0.006420,1.000000,0.402289,-0.007597
efficiency,-0.056452,0.019398,-0.071339,0.402289,1.000000,-0.049250
heading,0.041264,-0.022829,0.121177,-0.007597,-0.049250,1.000000


In [251]:
import argparse
import joblib
import os
import numpy as np
import glob

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score



In [401]:
dataset = pd.read_csv('./dataset.csv')
#dataset = dataset.loc[dataset['vessel'] == 2]
dataset['time_spent'] = dataset['time_spent'].apply(lambda x: float(x[0:2]))
dataset = dataset.drop(columns=['Unnamed: 0', 'begin_date', 'end_date'])
dataset.dropna(inplace=True)
X = dataset.drop('end_port_id', axis=1)
y = dataset['end_port_id']

In [402]:
from sklearn.model_selection import train_test_split
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

In [403]:
from sklearn.ensemble import RandomForestClassifier
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [404]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc_cv_score = cross_val_score(rfc, X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [405]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[ 1  0  0 ...  0  0  0]
 [ 0 18  0 ...  0  0  0]
 [ 0  0 17 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]]


=== Classification Report ===
              precision    recall  f1-score   support

         6.0       1.00      1.00      1.00         1
         7.0       0.82      1.00      0.90        18
        10.0       0.63      0.57      0.60        30
        11.0       0.67      0.62      0.64        13
        13.0       0.82      1.00      0.90        14
        14.0       0.89      1.00      0.94         8
        15.0       0.00      0.00      0.00         1
        19.0       1.00      0.80      0.89         5
        20.0       1.00      0.50      0.67         6
        21.0       0.90      1.00      0.95         9
        22.0       0.77      0.81      0.79        21
        23.0       1.00      0.25      0.40         4
        27.0       1.00      0.60      0.75         5
        29.0       0.75      0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [412]:
from sklearn.model_selection import RandomizedSearchCV

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']
# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 18.6min finished


{'n_estimators': 1600, 'max_features': 'auto', 'max_depth': 500}


In [414]:
rfc = RandomForestClassifier(n_estimators=1600, max_depth=500, max_features='auto')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10)
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


=== Confusion Matrix ===
[[ 1  0  0 ...  0  0  0]
 [ 0 18  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]]


=== Classification Report ===
              precision    recall  f1-score   support

         6.0       1.00      1.00      1.00         1
         7.0       0.82      1.00      0.90        18
        10.0       0.72      0.70      0.71        30
        11.0       0.67      0.62      0.64        13
        13.0       0.88      1.00      0.93        14
        14.0       0.89      1.00      0.94         8
        15.0       0.00      0.00      0.00         1
        19.0       1.00      0.80      0.89         5
        20.0       1.00      0.50      0.67         6
        21.0       0.90      1.00      0.95         9
        22.0       0.79      0.90      0.84        21
        23.0       1.00      0.25      0.40         4
        27.0       1.00      0.40      0.57         5
        29.0       0.67      0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [416]:
len(rfc_predict)

680